In [1]:
from cordra import CordraClient, Dataset, CordraObject
import json
from fastjsonschema import JsonSchemaValueException
import numpy as np
from io import BytesIO

In [44]:
host = 'https://dev.materialhub.org/'
creds = 'credentials.json'

materialhub = CordraClient(host=host, credentials_file=creds, verify=False)

m = np.load('m.npy')
p = np.load('p.npy')

In [3]:
r = materialhub.find('/username:amann37')

me = r['results'][0]
my_id = me['@id']
me

{'@context': ['http://schema.org/docs/jsonldcontext.json',
  'https://materialhub.org/devdocs/matContext.jsonld'],
 '@type': 'Person',
 '@id': 'prefix/68e2d0b811a8274e0ffc',
 'username': 'amann37',
 'password': '',
 'id': ''}

In [4]:
myObjects = materialhub.find(f'/metadata/createBy:_"{my_id}"', full=True)['results']
print('\n'.join([str(r.keys()) for r in myObjects]))

dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata', 'payloads'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_keys(['id', 'type', 'content', 'metadata'])
dict_key

In [45]:
dataset = Dataset(client=materialhub)

ds = dataset.add('Dataset', properties={'name':'2_phase_elastic_CR50'})

In [46]:
#defining terms for the properties
#TODO add definitions and more specifics
YM_term = dataset.add('DefinedTerm', properties={'name':'Young\'s Modulus'})
PR_term = dataset.add('DefinedTerm', properties={'name':'Poisson Ratio'})
effective_property = dataset.add('DefinedTerm', properties={'name':'Effective Property'})
broad_mat_term = dataset.add('DefinedTerm', properties={'name':'Two Phase Material'})

#defining the material properties by using the DF, values, and units
m1_ym = dataset.add('MaterialProperty', properties={'PropertyID':YM_term, 'name':YM_term.properties['name'], 'value':'120'})
m1_pr = dataset.add('MaterialProperty', properties={'PropertyID':PR_term, 'name':PR_term.properties['name'], 'value':'0.3'})
                                                                          
m2_ym = dataset.add('MaterialProperty', properties={'PropertyID':YM_term, 'name':YM_term.properties['name'], 'value':'6,000'})
m2_pr = dataset.add('MaterialProperty', properties={'PropertyID':PR_term, 'name':PR_term.properties['name'], 'value':'0.3'})

In [47]:
mat1 = dataset.add('Material', properties={'name':'Low Stiffness Material', 'about':[m1_ym.properties['@id'], m1_pr.properties['@id']]})
mat2 = dataset.add('Material', properties={'name':'High Stiffness Material', 'about':[m2_ym.properties['@id'], m2_pr.properties['@id']]})

In [48]:
broad_material = dataset.add('Material', properties={'PropertyID':broad_mat_term,'name':'2 Phase Material', 'about':[mat1.properties['@id'], mat2.properties['@id']]})

In [49]:
val = 1/10
eff_prop = dataset.add('MaterialProperty', properties={'PropertyID':effective_property, 'name':effective_property.properties['name']+f'={val}', 'value':f'{val}'})
narrow_mat_term = dataset.add('DefinedTerm', properties={'name':f'Two Phase - Effective stiffness: {val}', 'broader':[broad_mat_term.properties['@id']]})
                                                       
narrow_material = dataset.add('Material', properties={'PropertyID':narrow_mat_term, 'name':narrow_mat_term.properties['name'], 'about':[mat1.properties['@id'], mat2.properties['@id'], eff_prop.properties['@id']]})

True

In [8]:
structure = dataset.add('MaterialStructure', properties={'name':'test structure','material':[broad_material.properties['@id'], narrow_material.properties['@id']]}, payloads={'structure':BytesIO(m).getvalue()})

In [11]:
stress_field = dataset.add('DefinedTerm', properties={'name':'Stress_Field', 'description':'An array of stress states in each voxel of your domain'})
strain_field = dataset.add('DefinedTerm', properties={'name':'Strain_Field'})

stress_result = dataset.add('MaterialProperty', properties={'propertpyID':stress_field, 'material':[broad_material.properties['@id'], narrow_material.properties['@id']]}, payloads={'value':BytesIO(p).getvalue()})
strain_result = dataset.add('MaterialProperty', properties={'propertpyID':strain_field, 'material':[broad_material.properties['@id'], narrow_material.properties['@id']]}, payloads={'value':BytesIO(p).getvalue()})

In [13]:
#define abaqus
abaqus_term = dataset.add('DefinedTerm', properties={'name':'abaqus', 'description':'Finite Element Solver'})
abaqus = dataset.add('SoftwareApplication', properties={'propertyID':abaqus_term})


fem = dataset.add('Action', properties={'agent':[abaqus.properties['@id']], 'object':[structure.properties['@id']], 'result':[stress_result.properties['@id'], strain_result.properties['@id']]})


#define python
python_term = dataset.add('DefinedTerm', properties={'name':'python', 'description':'programming language'})
python = dataset.add('SoftwareApplication', properties={'propertyID':python_term})

eff_stiff = dataset.add('Action', properties={'agent':[python.properties['@id']], 'object':[stress_result.properties['@id'], strain_result.properties['@id']], 'result':[narrow_material.properties['@id']] })





In [35]:
import h5py
import os

cwd = os.getcwd()

train_r = os.path.join(cwd, '..', '..', '..', 'Elastic_Localization_Data', 'cr_50_full', '31_c50_train_responses.h5')
train_m = os.path.join(cwd, '..', '..', '..', 'Elastic_Localization_Data', 'cr_50_full', '31_c50_train_micros.h5')

train_m = h5py.File(train_m)
train_r = h5py.File(train_r)


m = train_m['micros']
train_stress = train_r['stress']
train_strain = train_r['strain']

avg_stress = np.sum(train_stress, axis=1) / 31**3
avg_strain = np.sum(train_strain, axis=1) / 31**3

dp = len(avg_stress)

In [37]:
avg_stress[0]

array([ 3.32984945,  0.90224451,  0.86666594,  0.04540909, -0.08411627,
        0.05960775])

In [40]:
eff_p = (avg_stress[0] / avg_strain[0])[0].round()
eff_p

3330.0

In [32]:

for i in range(dp):
    eff_p = (avg_stress / avg_strain)[:,0].round()

In [33]:
eff_p

array([3330., 1963., 3292., 1175.,  180.,  573., 1932., 2797.,  182.,
       3015.])

In [34]:

print(dp)

10
